In [11]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import execute, Aer
# from qc_grader import prepare_ex3, grade_ex3
import numpy as np
from heapq import nlargest
import time


In [12]:

problem_set = [
    [["0", "2"], ["1", "0"], ["1", "2"], ["1", "3"], ["2", "0"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "2"], ["2", "2"], ["3", "0"], ["3", "3"]],
    [["0", "0"], ["1", "1"], ["1", "3"], ["2", "0"], ["3", "2"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "1"], ["1", "3"], ["3", "2"], ["3", "3"]],
    [["0", "2"], ["1", "0"], ["1", "3"], ["2", "0"], ["3", "2"], ["3", "3"]],
    [["1", "1"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "1"], ["3", "3"]],
    [["0", "2"], ["0", "3"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "3"]],
    [["0", "0"], ["0", "3"], ["1", "2"], ["2", "2"], ["2", "3"], ["3", "0"]],
    [["0", "3"], ["1", "1"], ["1", "2"], ["2", "0"], ["2", "1"], ["3", "3"]],
    [["0", "0"], ["0", "1"], ["1", "3"], ["2", "1"], ["2", "3"], ["3", "0"]],
    [["0", "1"], ["0", "3"], ["1", "2"], ["1", "3"], ["2", "0"], ["3", "2"]],
    [["0", "0"], ["1", "3"], ["2", "0"], ["2", "1"], ["2", "3"], ["3", "1"]],
    [["0", "1"], ["0", "2"], ["1", "0"], ["1", "2"], ["2", "2"], ["2", "3"]],
    [["0", "3"], ["1", "0"], ["1", "3"], ["2", "1"], ["2", "2"], ["3", "0"]],
    [["0", "2"], ["0", "3"], ["1", "2"], ["2", "3"], ["3", "0"], ["3", "1"]],
    [["0", "1"], ["1", "0"], ["1", "2"], ["2", "2"], ["3", "0"], ["3", "1"]],
]

In [13]:
import numpy as np
from itertools import permutations
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer, execute
import time


def diffusion(qc: QuantumCircuit, qubits: QuantumRegister, qr_extra: QuantumRegister):

    qc.h(qubits)
    qc.x(qubits)
    qc.h(qubits[-1])
    try:
        qc.mct(qubits[:-1], qubits[-1], qr_extra, mode="v-chain")
    except ValueError:
        qc.mct(qubits[:-1], qubits[-1], qr_extra, mode="recursion")
    qc.h(qubits[-1])
    qc.x(qubits)
    qc.h(qubits)

    qc.barrier()

    return qc


def mark_address(qc: QuantumCircuit, qubits: QuantumRegister, index: int):

    if index not in range(2 ** len(qubits)):
        raise ValueError(f"index is {index} must be in range(16)")

    # if len(qubits) != 4:
    #    raise ValueError("qubits must have length of 4")

    # Convert index to a binary string
    if len(qubits) == 1:
        bin_str = f"{index:01b}"
    if len(qubits) == 2:
        bin_str = f"{index:02b}"
    if len(qubits) == 4:
        bin_str = f"{index:04b}"

    for i in range(len(qubits)):
        if bin_str[i] == "0":
            qc.x(qubits[i])

    return qc


def game_logic_oracle(
    qr_shots: QuantumRegister,
    qr_address: QuantumRegister,
    qr_cluster: QuantumRegister,
    qr_extra: QuantumRegister,
    prob_set,
    return_gate: bool = True,
):

    qc = QuantumCircuit(qr_shots, qr_address, qr_cluster, qr_extra)

    for p, puzzle in enumerate(prob_set):  # for each puzzle

        puzzle = np.array(puzzle, int)  # convert to int
        # Mark the puzzle's address (p):
        qc = mark_address(qc, qr_address, p)

        for c, (h, v) in enumerate(puzzle):  # for each position pair in puzzle
            # Control cluster c with address and the puzzle's horizontal number
            qc.mct(
                qr_address[:] + [qr_shots[h]], qr_cluster[c], qr_extra, mode="v-chain",
            )
            qc.mct(
                qr_address[:] + [qr_shots[v + 4]],
                qr_cluster[c],
                qr_extra,
                mode="v-chain",
            )

        # Unmark the puzzles's address:
        qc = mark_address(qc, qr_address, p)

    # Convert to gate and return
    if return_gate:

        game_logic = qc.to_gate()
        game_logic.name = "Game Logic"
        return game_logic
    else:
        return qc


def row_to_dict(row):

    row_dict = {}
    for i, item in enumerate(row):

        if len(row) == 1:
            bin_str = f"{i:01b}"
        if len(row) == 2:
            bin_str = f"{i:01b}"
        if len(row) == 4:
            bin_str = f"{i:02b}"
        if len(row) == 8:
            bin_str = f"{i:02b}"
        if len(row) == 16:
            bin_str = f"{i:04b}"

        row_dict[bin_str] = item

    return row_dict


def get_qram_logic_score(cluster_dict, bit_string="", branch_order=[3, 2, 1, 0]):

    score_array = [10, 69, 223, 487, 1015]

    # Check to see if there is only one value in the dictionary.values()
    if len(set(cluster_dict.values())) == 1:
        # Unique_value = set(cluster_dict.values()).pop()
        # print(f"We have a leaf with value {unique_value} with bit_string {bit_string}")
        return score_array[len(bit_string)]

    zero_dict = {}
    one_dict = {}
    current_bit = branch_order[0]

    for k, v in cluster_dict.items():
        if k[current_bit] == "0":
            zero_dict[k] = v
        else:
            one_dict[k] = v

    score1 = get_qram_logic_score(zero_dict, bit_string + "0", branch_order[1:])
    score2 = get_qram_logic_score(one_dict, bit_string + "1", branch_order[1:])

    return score1 + score2


def get_lowest_score_branching_order(row_of_positions):
    best_score = 9999999
    best_perm = None

    for p in permutations([0, 1, 2, 3]):
        score = get_qram_logic_score(row_to_dict(row_of_positions), branch_order=p)
        if score < best_score:
            best_score = score
            best_perm = p

        return best_perm, best_score


def load_data_with_recursion(
    qc,
    target_cluster,
    control_address,
    control_shots,
    extra_qubits,
    cluster_dict,
    bit_string="",
    branch_order=[0, 1, 2, 3],
):

    depth = len(bit_string)

    # Check to see if there is only one value in the dictionary.values()
    if len(set(cluster_dict.values())) == 1:
        shot_index = set(cluster_dict.values()).pop()
        # print(f"We have a leaf with value {unique_value} with bit_string {bit_string}")
        depth = len(bit_string)

        # Mark address qubits
        for i in range(depth):
            if bit_string[i] == "0":
                address_index = branch_order[i]
                qc.x(control_address[address_index])

        if depth == 0:
            # All problems have same value. CNOT will suffice
            qc.cx(control_shots[shot_index], target_cluster)
        elif depth == 1:
            # One address, one shot, are in control
            qc.ccx(
                control_shots[shot_index],
                control_address[branch_order[0]],
                target_cluster,
            )
        elif depth > 1:
            addr_indices = branch_order[0:depth]
            control_qubits = [control_shots[shot_index]]
            for index in addr_indices:
                control_qubits += [control_address[index]]
            qc.mct(control_qubits, target_cluster, extra_qubits, mode="v-chain")

        # De-mark address qubits
        for i in range(depth):
            if bit_string[i] == "0":
                address_index = branch_order[i]
                qc.x(control_address[address_index])

        return qc

    zero_dict = {}
    one_dict = {}
    current_bit = branch_order[depth]

    for k, v in cluster_dict.items():
        if k[current_bit] == "0":
            zero_dict[k] = v
        else:
            one_dict[k] = v

    qc = load_data_with_recursion(
        qc=qc,
        target_cluster=target_cluster,
        control_address=control_address,
        control_shots=control_shots,
        extra_qubits=extra_qubits,
        cluster_dict=zero_dict,
        bit_string=bit_string + "0",
        branch_order=branch_order,
    )
    qc = load_data_with_recursion(
        qc=qc,
        target_cluster=target_cluster,
        control_address=control_address,
        control_shots=control_shots,
        extra_qubits=extra_qubits,
        cluster_dict=one_dict,
        bit_string=bit_string + "1",
        branch_order=branch_order,
    )

    return qc


def game_logic_oracle_opt(
    qr_shots: QuantumRegister,
    qr_address: QuantumRegister,
    qr_cluster: QuantumRegister,
    qr_extra: QuantumRegister,
    prob_set,
    return_gate: bool = True,
):

    qc = QuantumCircuit(qr_shots, qr_address, qr_cluster, qr_extra)

    ps = np.array(prob_set, int)
    cluster_horz = ps.T[0]
    cluster_vert = ps.T[1]

    for i, cluster in enumerate(cluster_horz):

        # Identify optimal branching order for this cluster
        cluster_dict = row_to_dict(cluster)
        perm, _ = get_lowest_score_branching_order(cluster_dict)

        qc = load_data_with_recursion(
            qc=qc,
            target_cluster=qr_cluster[i],
            control_address=qr_address,
            control_shots=qr_shots[0:4],
            extra_qubits=qr_extra,
            cluster_dict=cluster_dict,
            branch_order=perm,
        )

    for i, cluster in enumerate(cluster_vert):

        # Identify optimal branching order for this cluster
        cluster_dict = row_to_dict(cluster)
        perm, _ = get_lowest_score_branching_order(cluster_dict)

        qc = load_data_with_recursion(
            qc=qc,
            target_cluster=qr_cluster[i],
            control_address=qr_address,
            control_shots=qr_shots[4:8],
            extra_qubits=qr_extra,
            cluster_dict=cluster_dict,
            branch_order=perm,
        )

    # Convert to gate and return
    if return_gate:

        game_logic = qc.to_gate()
        game_logic.name = "Game Logic"
        return game_logic
    else:
        return qc


In [14]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, execute, Aer
import numpy as np


def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    circ = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n // 2):
        circ.swap(qubit, n - qubit - 1)
    for j in range(n):
        for m in range(j):
            circ.cp(-np.pi / float(2 ** (j - m)), m, j)
        circ.h(j)

    qft_dag = circ.to_gate()
    qft_dag.name = "$QFT_dag$"
    return qft_dag


def count_oracle(
    qr_count_me: QuantumRegister, qr_counter: QuantumRegister, return_gate: bool = True,
):
    """ Creates a gate/circuit that counts the number of 1's in
    qr_count_me and writes to qr_counter

    ----------
    PARAMETERS
    ----------
    qr_count_me:
        A quantum register whose 1's will be counted
    qr_counter:
        A quantum register that will record the count
    return_gate:
        Returns gate if true, quantum circuit if false.

    -------
    RETURNS
    -------
    A gate or quantum circuit that counts qr_count_me and writes to qr_counter

    """

    qc = QuantumCircuit(qr_count_me, qr_counter)

    # Step 1: Put the counter into superposition
    qc.h(qr_counter)

    for qubit_count_me in qr_count_me:
        for i, qubit_counter in enumerate(qr_counter):
            # For each less significant qubit, we need to do a
            # smaller-angled controlled rotation:
            qc.cp(np.pi / 2 ** (len(qr_counter) - i - 1), qubit_count_me, qubit_counter)

    qc.append(qft_dagger(len(qr_counter)), qargs=qr_counter)

    if return_gate:
        counter_gate = qc.to_gate()
        counter_gate.name = "Counter"
        return counter_gate
    else:
        return qc


if __name__ == "__main__":

    count_me_size = 8
    counter_size = 4

    qr_count_me = QuantumRegister(count_me_size)
    qr_counter = QuantumRegister(counter_size)

    cr_count_me = ClassicalRegister(count_me_size)
    cr_counter = ClassicalRegister(counter_size)

    qc = QuantumCircuit(qr_count_me, qr_counter, cr_count_me, cr_counter)

    # Put qr_count_me into superposition
    qc.h(qr_count_me)

    counter_gate = count_oracle(
        qr_count_me=qr_count_me, qr_counter=qr_counter, return_gate=True
    )

    qc.append(
        counter_gate, qr_count_me[:] + qr_counter[:],
    )

    qc.measure(qr_count_me, cr_count_me)
    qc.measure(qr_counter, cr_counter)

    backend = Aer.get_backend("qasm_simulator")
    job = execute(qc, backend=backend, shots=2000,)

    result = job.result()
    count = result.get_counts()

    for k, v in count.items():
        print(k, v)


0000 00000000 12
0001 00000001 5
0001 00000010 6
0001 00000100 7
0001 00001000 5
0001 00010000 11
0001 00100000 4
0001 01000000 4
0001 10000000 13
0010 00000011 7
0010 00000101 15
0010 00000110 3
0010 00001001 10
0010 00001010 4
0010 00001100 10
0010 00010001 2
0010 00010010 6
0010 00010100 8
0010 00011000 12
0010 00100001 7
0010 00100010 6
0010 00100100 7
0010 00101000 3
0010 00110000 6
0010 01000001 7
0010 01000010 6
0010 01000100 6
0010 01001000 6
0010 01010000 6
0010 01100000 7
0010 10000001 10
0010 10000010 10
0010 10000100 5
0010 10001000 7
0010 10010000 4
0010 10100000 9
0010 11000000 9
0011 00000111 8
0011 00001011 5
0011 00001101 4
0011 00001110 6
0011 00010011 10
0011 00010101 8
0011 00010110 7
0011 00011001 6
0011 00011010 10
0011 00011100 13
0011 00100011 8
0011 00100101 7
0011 00100110 4
0011 00101001 8
0011 00101010 7
0011 00101100 8
0011 00110001 9
0011 00110010 6
0011 00110100 4
0011 00111000 11
0011 01000011 3
0011 01000101 6
0011 01000110 6
0011 01001001 6
0011 010010

In [16]:

def week3_ans_func(prob_set, count_shots=False):

    num_problems = len(prob_set)
    address_qubits = int(np.sqrt(num_problems))
    counting_qubits = 4

    # 4 + 4 (shot options) + 6 (clusters) + 4 (address) + 3 (counting) + 1 (ancilla) = 22, 6 extra!
    qr_shots = QuantumRegister(8, name="shots")
    qr_cluster = QuantumRegister(6, name="clusters")
    qr_address = QuantumRegister(address_qubits, name="address")
    qr_counter = QuantumRegister(counting_qubits, name="counting")
    qr_ancilla = QuantumRegister(2, name="ancilla")
    num_extra = 28 - len(
        qr_shots[:] + qr_cluster[:] + qr_address[:] + qr_counter[:] + qr_ancilla[:]
    )

    qr_extra = QuantumRegister(num_extra, name="extra")

    cr_shots = ClassicalRegister(len(qr_shots))
    cr_address = ClassicalRegister(address_qubits)

    if count_shots:
        qc = QuantumCircuit(
            qr_shots,
            qr_address,
            qr_cluster,
            qr_counter,
            qr_ancilla,
            qr_extra,
            cr_shots,
        )
    else:
        qc = QuantumCircuit(
            qr_shots,
            qr_address,
            qr_cluster,
            qr_counter,
            qr_ancilla,
            qr_extra,
            cr_address,
        )

    # Set cluster status to 0
    # No code required

    # Prepare ancilla
    qc.x(qr_ancilla)
    qc.h(qr_ancilla)

    # Put solution into superposition
    qc.h(qr_shots[:] + qr_address[:])

    qc.barrier()

    # Load game gate
    game_qubits = qr_shots[:] + qr_address[:] + qr_cluster[:] + qr_extra[:]

    game_gate = game_logic_oracle_opt(
        qr_shots=qr_shots,
        qr_address=qr_address,
        qr_cluster=qr_cluster,
        qr_extra=qr_extra,
        prob_set=prob_set,
    )

    # Load counter gate
    counter_qubits = qr_shots[:] + qr_counter[:]
    counter_gate = count_oracle(
        qr_count_me=qr_shots, qr_counter=qr_counter, return_gate=True
    )

    game_iterations = 1
    # Code for Grover's algorithm with iterations = 1 will be as follows.
    for _ in range(1):

        # Perform Grover's algorithm for game logic
        for j in range(game_iterations):

            # Append game gate
            qc.append(game_gate, game_qubits)

            # Check solution
            try:
                qc.mct(qr_cluster, qr_ancilla[0], qr_extra, mode="v-chain")
            except ValueError:
                qc.mct(qr_cluster, qr_ancilla[0], qr_extra, mode="recursion")

            # Uncompute game gate
            qc.append(game_gate.inverse(), game_qubits)

            # Diffusion circuit
            qc = diffusion(qc, qr_shots, qr_extra)

        # Append counter gate
        qc.append(counter_gate, counter_qubits)

        # Mark the desired state: Exactly 4: 0100
        qc.x(qr_counter[0:2])
        if counting_qubits >= 4:
            qc.x(qr_counter[3])

        # Check for the solution
        #qc.h(qr_ancilla[1])
        qc.mct(qr_counter, qr_ancilla[0], qr_extra, mode="v-chain")
        #qc.h(qr_ancilla[1])

        # Unmark the desired state
        qc.x(qr_counter[0:2])
        if counting_qubits >= 4:
            qc.x(qr_counter[3])

        # Uncompute counter gate
        qc.append(counter_gate.inverse(), counter_qubits)

        # Reverse Grover's algorithm for game logic
        for j in range(game_iterations):

            # Diffusion circuit
            qc = diffusion(qc, qr_shots, qr_extra)

            # Append game gate
            qc.append(game_gate.inverse(), game_qubits)

            # Check solution
            try:
                qc.mct(qr_cluster, qr_ancilla[0], qr_extra, mode="v-chain")
            except ValueError:
                qc.mct(qr_cluster, qr_ancilla[0], qr_extra, mode="recursion")

            # Uncompute game gate
            qc.append(game_gate, game_qubits)

        # Diffusion circuit
        qc = diffusion(qc, qr_address, qr_extra)

    # Measure results
    if count_shots:
        qc.measure(qr_shots, cr_shots[::-1])
    else:
        qc.measure(qr_address, cr_address)

    return qc


In [18]:
from test_data import q1, q2, q3

qc = week3_ans_func(q2[0], count_shots=False)

backend = Aer.get_backend("qasm_simulator")
tic = time.perf_counter()
job = execute(
    qc,
    backend=backend,
    shots=1000,
    optimization_level=3,
    backend_options={"fusion_enable": True},
)

result = job.result()
count = result.get_counts()

for k, v in count.items():
    print(k, v)

toc = time.perf_counter()

print(f"Circuit executed in {toc - tic:0.4f} seconds")

In [15]:
# Submission code
from qc_grader import grade_ex3, prepare_ex3, submit_ex3

# Execute your circuit with following prepare_ex3() function.
# The prepare_ex3() function works like the execute() function with only QuantumCircuit as an argument.
job = prepare_ex3(week3_ans_func)

result = job.result()
counts = result.get_counts()
original_problem_set_counts = counts[0]

original_problem_set_counts
# The bit string with the highest number of observations is treated as the solution.

In [ ]:
grade_ex3(job)